In [190]:
%run './remodelisations/communes.ipynb'
%run './remodelisations/dep.ipynb'
%run './remodelisations/regions.ipynb'
# %run './remodelisations/statistiques.ipynb' # les stats ont des problèmes avec les fk -> ref vers des code de communes inconnues


In [321]:
import psycopg2 as postgres

In [322]:
BASE = 'tp_dba'
USER = 'axelmoriceau'
PSWD = ''

conn = postgres.connect(
    host='localhost',
    database=BASE,
    user=USER,
    password=PSWD,
    port='5432'
)

cur = conn.cursor()


In [323]:
years = ['1968', '1975', '1982', '1990', '1999', '2008', '2013', '2018']
stats = []
for year in years:
    stats.append(f'stats_{year}')


In [324]:
# A enlever avant rendu

drop_stats = ''
for stat in stats:
    drop_stats += f'DROP TABLE IF EXISTS {stat};'

In [325]:
tables = ['departements_cheflieu','regions_cheflieu', 'communes', 'departements', 'regions']

In [326]:
# A enlever avant rendu 

drop_tables = ''
for table in tables:
    drop_tables += f'DROP TABLE IF EXISTS {table};'


In [327]:
cur.execute(drop_stats)
cur.execute(drop_tables)
conn.commit()

In [328]:
cur.execute(open("sql/create.sql", "r").read())
conn.commit()

In [329]:
years = ['1968', '1975', '1982', '1990', '1999', '2008', '2013', '2018']

files = [
        'regions', 
        'departements',
        'communes',
        'departements_cheflieu',
        'regions_cheflieu',
    ] + stats

for file in files:
    path = f'./csv/filtered/{file}.csv'
    cur.copy_from(open(path, 'r'), file, sep=',')
    conn.commit()


In [330]:
conn.close()